In [1]:
import os
import pandas as pd
import re
from scipy.stats.mstats import gmean

In [2]:
# get input paths
test_set = "miplib_smaller_easy_binary"
instance_fldr = os.path.join("test_sets", test_set)
results_fldr = os.path.join("results", test_set)
out_fldr = os.path.join("outputs", test_set)

## Check run failures

In [3]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "walltime": [],
    "bad_alloc": [],
    "takeoffcuts": [],
    "solver is dual infeasible": [],
    "solver must be optimal": [],
    "segmentation fault": [],
    "no vpcs were made from a new disjunction": [],
    "farkas": [],
    "prlp is primal infeasible": [],
    "branch variables must be integer": [],
    "warning": []
}

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# series that didn't run
no_go = []

# counts
count_series = 0
count_instances = 0

# iterate over all expected runs
for instance in os.listdir(instance_fldr):
    if not os.path.isdir(os.path.join(instance_fldr, instance)):
        continue
    for perturbation in os.listdir(os.path.join(instance_fldr, instance)):
        if not os.path.isdir(os.path.join(instance_fldr, instance, perturbation)):
            continue
        for terms in [4, 16, 64]:
            for generator in ["None", "New", "Old", "Farkas"]:

                # set variables for this iteration
                count_series += 1
                stem = f"{instance}_{perturbation}_{terms}_{generator}"
                file_pth = os.path.join(results_fldr, f"{stem}.err")
                series_fldr = os.path.join(instance_fldr, instance, perturbation)
                count_instances += len([f for f in os.listdir(series_fldr) if
                                        f.endswith(".mps")])

                # check if the series wasn't run
                if not os.path.exists(file_pth):
                    no_go.append(stem)
                
                # check if the series ran with no errors or warnings
                elif os.path.getsize(file_pth) == 0:
                    empty.append(stem)
                
                # track which error codes were thrown
                else:
                    # read the file
                    with open(file_pth, "r") as f:
                        text = f.read().lower()
                    
                    # assign the error file to the appropriate list
                    for code in err:
                        if code in text:
                            err[code].append(stem)
                            break
                    else:
                        other.append(stem)

In [4]:
# check which series didn't run
print(no_go)

[]


In [5]:
# get the proportion of series that at least got started
1 - (len(no_go) / count_series)

1.0

In [6]:
err["walltime"]

['neos-3610173-itata_matrix_1_64_New', 'neos-3610173-itata_matrix_0_64_New']

In [7]:
err["bad_alloc"]

['neos-3610173-itata_rhs_0_64_Old',
 'neos-3610173-itata_rhs_0_64_Farkas',
 'neos-3610173-itata_rhs_1_64_Old',
 'neos-3610173-itata_rhs_1_64_Farkas',
 'neos-3610173-itata_matrix_1_64_Old',
 'neos-3610173-itata_matrix_1_64_Farkas',
 'neos-3610173-itata_matrix_0_64_Old',
 'neos-3610173-itata_matrix_0_64_Farkas',
 'neos-3610051-istra_rhs_0_64_Old',
 'neos-3610051-istra_rhs_0_64_Farkas',
 'neos-3610051-istra_rhs_1_64_Old',
 'neos-3610051-istra_rhs_1_64_Farkas',
 'neos-3610051-istra_matrix_1_64_Old',
 'neos-3610051-istra_matrix_1_64_Farkas',
 'neos-3610051-istra_matrix_0_64_Old',
 'neos-3610051-istra_matrix_0_64_Farkas',
 'neos-3611447-jijia_rhs_0_64_Old',
 'neos-3611447-jijia_rhs_0_64_Farkas',
 'neos-3611447-jijia_rhs_1_64_Old',
 'neos-3611447-jijia_rhs_1_64_Farkas',
 'neos-3611447-jijia_matrix_1_64_Old',
 'neos-3611447-jijia_matrix_1_64_Farkas',
 'neos-3611447-jijia_matrix_0_64_Old',
 'neos-3611447-jijia_matrix_0_64_Farkas',
 'neos-3610040-iskar_rhs_1_64_Old',
 'neos-3610040-iskar_rhs_1_6

In [8]:
err["takeoffcuts"]

['dcmulti_rhs_-1_4_Old',
 'dcmulti_rhs_-1_4_Farkas',
 'dcmulti_rhs_-1_16_Farkas',
 'dcmulti_rhs_-1_64_Farkas',
 'dcmulti_matrix_-1_4_Old',
 'dcmulti_matrix_-1_16_Old',
 'dcmulti_matrix_-1_64_Old']

In [9]:
err["solver is dual infeasible"]

[]

In [10]:
err["solver must be optimal"]

['neos-5140963-mincio_objective_1_4_New',
 'neos-5140963-mincio_objective_1_4_Old',
 'neos-5140963-mincio_objective_1_4_Farkas',
 'neos-5140963-mincio_objective_1_16_New',
 'neos-5140963-mincio_objective_1_16_Old',
 'neos-5140963-mincio_objective_1_16_Farkas',
 'neos-5140963-mincio_objective_1_64_New',
 'neos-5140963-mincio_objective_0_4_New',
 'neos-5140963-mincio_objective_0_4_Old',
 'neos-5140963-mincio_objective_0_4_Farkas',
 'neos-5140963-mincio_objective_0_16_New',
 'neos-5140963-mincio_objective_0_16_Old',
 'neos-5140963-mincio_objective_0_16_Farkas',
 'neos-5140963-mincio_objective_0_64_New',
 'neos-5140963-mincio_objective_0_64_Old',
 'neos-5140963-mincio_objective_0_64_Farkas',
 'neos-5140963-mincio_objective_-1_4_New',
 'neos-5140963-mincio_objective_-1_4_Old',
 'neos-5140963-mincio_objective_-1_4_Farkas',
 'neos-5140963-mincio_objective_-1_16_New',
 'neos-5140963-mincio_objective_-1_16_Old',
 'neos-5140963-mincio_objective_-1_16_Farkas',
 'neos-5140963-mincio_objective_-1_6

In [11]:
err["segmentation fault"]

[]

In [12]:
err["no vpcs were made from a new disjunction"]

['neos-3610173-itata_rhs_0_64_New',
 'neos-3610051-istra_rhs_1_64_New',
 'mas74_rhs_1_4_New',
 'mas74_rhs_1_16_New',
 'mas74_rhs_1_64_New',
 'mas74_matrix_1_4_New',
 'mas74_matrix_1_16_New',
 'mas74_matrix_1_64_New',
 'mas74_matrix_0_4_New',
 'mas74_matrix_0_16_New',
 'mas74_matrix_0_64_New',
 'mas74_matrix_-1_4_New',
 'mas74_matrix_-1_16_New',
 'mas74_matrix_-1_64_New',
 'neos-5140963-mincio_rhs_0_64_New',
 'bm23_rhs_0_64_New',
 'bm23_rhs_1_64_New',
 'bm23_matrix_1_64_New',
 'bm23_matrix_0_64_New',
 'bm23_matrix_-1_64_New',
 'dcmulti_matrix_1_4_New',
 'dcmulti_matrix_1_16_New',
 'dcmulti_matrix_1_64_New',
 'dcmulti_matrix_0_4_New',
 'dcmulti_matrix_0_16_New',
 'dcmulti_matrix_0_64_New',
 'dcmulti_matrix_-1_64_New',
 'mas76_matrix_1_4_New',
 'mas76_matrix_1_16_New',
 'mas76_matrix_1_64_New',
 'mas76_matrix_0_4_New',
 'mas76_matrix_0_16_New',
 'mas76_matrix_0_64_New',
 'mas76_matrix_-1_64_New',
 'prod1_rhs_0_4_New',
 'prod1_rhs_1_4_New']

In [14]:
err["farkas"]

['neos-3610173-itata_rhs_0_16_New',
 'neos-3610173-itata_rhs_1_64_New',
 'neos-3610051-istra_rhs_1_16_New',
 'neos-3610051-istra_matrix_1_16_New',
 'dcmulti_objective_1_64_New']

In [15]:
err["prlp is primal infeasible"]

['mas74_matrix_1_4_Old',
 'mas74_matrix_1_16_Old',
 'mas74_matrix_1_64_Old',
 'neos-5140963-mincio_matrix_1_4_Old',
 'dcmulti_matrix_1_4_Old',
 'dcmulti_matrix_1_16_Old',
 'dcmulti_matrix_0_4_Old',
 'dcmulti_matrix_0_16_Old',
 'mas76_rhs_0_4_Old',
 'mas76_rhs_1_4_Old',
 'mas76_matrix_1_4_Old',
 'mas76_matrix_1_16_Old',
 'mas76_matrix_1_64_Old',
 'mas76_matrix_0_4_Old',
 'mas76_matrix_0_16_Old',
 'mas76_matrix_0_64_Old',
 'mas76_matrix_-1_4_Old',
 'mas76_matrix_-1_64_Old']

In [16]:
err["warning"]

['neos-3610173-itata_rhs_1_4_New',
 'neos-3610173-itata_rhs_1_16_New',
 'neos-5140963-mincio_matrix_1_4_New',
 'neos-5140963-mincio_matrix_1_16_New',
 'neos-5140963-mincio_matrix_1_64_New',
 'neos-5140963-mincio_matrix_0_4_New',
 'neos-5140963-mincio_matrix_0_16_New',
 'neos-5140963-mincio_matrix_0_64_New',
 'neos-3611689-kaihu_matrix_1_4_New',
 'neos-3611689-kaihu_matrix_1_16_New',
 'neos-3611689-kaihu_matrix_1_64_New',
 'dcmulti_objective_1_4_New',
 'dcmulti_objective_1_16_New',
 'dcmulti_objective_0_4_New',
 'dcmulti_objective_0_16_New',
 'dcmulti_objective_0_64_New',
 'dcmulti_rhs_-1_4_New',
 'dcmulti_rhs_-1_16_New',
 'dcmulti_rhs_-1_64_New',
 'dcmulti_matrix_-1_4_New',
 'dcmulti_matrix_-1_16_New',
 'mas76_matrix_-1_16_Old',
 'prod1_rhs_0_4_Old',
 'prod1_rhs_0_4_Farkas',
 'prod1_rhs_1_4_Old',
 'prod1_rhs_1_4_Farkas',
 'prod1_rhs_1_16_New',
 'prod1_rhs_1_16_Old',
 'prod1_rhs_1_16_Farkas',
 'prod1_matrix_1_4_New',
 'prod1_matrix_1_4_Old',
 'prod1_matrix_1_4_Farkas',
 'prod1_matrix_1_

In [17]:
err["branch variables must be integer"]

['prod1_rhs_0_16_New',
 'prod1_rhs_0_64_New',
 'prod1_rhs_0_64_Old',
 'prod1_rhs_0_64_Farkas',
 'prod1_rhs_1_64_New',
 'prod1_rhs_1_64_Old',
 'prod1_rhs_1_64_Farkas',
 'prod1_matrix_1_64_New',
 'prod1_matrix_1_64_Old',
 'prod1_matrix_1_64_Farkas',
 'prod1_matrix_0_64_New',
 'prod1_matrix_0_64_Old',
 'prod1_matrix_0_64_Farkas']

In [18]:
# errors unaccounted for
other

[]

In [19]:
# proportion of series that had no issues
len(empty + err["warning"]) / count_series

0.6863636363636364

In [20]:
# proportion of series that had no issues broken down by generator
for generator in ["None", "New", "Old", "Farkas"]:
    g = [s for s in empty if s.endswith(generator)] + \
        [s for s in err["warning"] if s.endswith(generator)]
    print(f"{generator}: {4 * len(g) / count_series}")

None: 1.0
New: 0.6545454545454545
Old: 0.48484848484848486
Farkas: 0.6060606060606061


In [21]:
# proportion of series that were improperly provisioned
(len(err["bad_alloc"] + err["walltime"])) / count_series

0.11363636363636363

In [22]:
(len(err["no vpcs were made from a new disjunction"] +
     err["prlp is primal infeasible"])) / count_series

0.08181818181818182

In [23]:
# proportion of series that had didn't actually error
(len(empty + err["bad_alloc"] + err["walltime"] + err["warning"] +
     err["no vpcs were made from a new disjunction"] +
     err["prlp is primal infeasible"])) / count_series

0.8818181818181818

## Read in data

In [24]:
# map generator names to the corresponding data frames
df_map = {
    "None": pd.DataFrame(),
    "Farkas": pd.DataFrame(),
    "New": pd.DataFrame(),
    "Old": pd.DataFrame()
}
regex = re.compile(r'([a-zA-Z0-9-]+)_([a-z]+)_([0-9-]+)_([0-9]+)_([a-zA-Z ]+)')

# iterate over all files in the folder
for file_name in os.listdir(results_fldr):
    
    file_pth = os.path.join(results_fldr, file_name)
    
    # if the file is not a nonempty csv, skip it
    if not file_name.endswith(".csv") or os.path.getsize(file_pth) == 0:
        continue
    
    # get the experimental set up
    match = regex.search(file_name)
    instance_name = match.group(1)
    perturbation = match.group(2)
    degree = 2**int(match.group(3))
    terms = int(match.group(4))
    generator = match.group(5)
    
    # read the file
    df = pd.read_csv(file_pth, keep_default_na=False)
    
    # add some identifying columns
    df["problem number"] = df.index
    df["instance"] = instance_name
    df["perturbation"] = perturbation
    df["degree"] = degree
    df["terms"] = terms
    
    # append to the appropriate data frame
    df_map[generator] = pd.concat([df_map[generator], df])

In [25]:
# get proportion of instances run
for generator, df in df_map.items():
    print(f"{generator}: {4 * len(df) / count_instances}")

None: 1.0
Farkas: 0.8173356602675974
New: 0.7201861547411286
Old: 0.8155904595695171


In [46]:
gen = "Farkas" # make sure masks 3 and 9 are fixed rest should be fine
masks = {
        0: -1e20 > df_map[gen]["lpBound"],
        1: df_map[gen]["lpBound"] - 1e-4 > df_map[gen]["lpBoundPostVpc"],
        2: df_map[gen]["lpBoundPostVpc"] - 1e-4 > df_map[gen]["disjunctiveDualBound"],
        3: df_map[gen]["lpBoundPostVpc"] - 1e-4 > df_map[gen]["rootDualBound"],
        4: df_map[gen]["rootDualBound"] - 1e-4 > df_map[gen]["dualBound"],
        5: df_map[gen]["dualBound"] - 1e-4 > df_map[gen]["primalBound"],
        6: df_map[gen]["primalBound"] > 1e20,
        7: 0 > df_map[gen]["vpcGenerationTime"],
        8: df_map[gen]["vpcGenerationTime"] - 1e-4 > df_map[gen]["heuristicTime"],
        9: df_map[gen]["heuristicTime"] - 1e-4 > df_map[gen]["rootDualBoundTime"],
        10: df_map[gen]["rootDualBoundTime"] - 1e-4 > df_map[gen]["terminationTime"],
        11: 0 > df_map[gen]["firstSolutionTime"],
        12: df_map[gen]["firstSolutionTime"] - 1e-4 > df_map[gen]["bestSolutionTime"],
        13: df_map[gen]["bestSolutionTime"] - 1e-4 > df_map[gen]["terminationTime"],
        14: df_map[gen]["terminationTime"] > df_map[gen]["maxTime"] + 10,
        15: df_map[gen]["vpcGenerationTime"] > df_map[gen]["maxTime"]
    }

df_map[gen][masks[9]]

,lpBound,disjunctiveDualBound,lpBoundPostVpc,rootDualBound,dualBound,heuristicPrimalBound,primalBound,vpcGenerationTime,heuristicTime,rootDualBoundTime,...,maxTime,vpcGenerator,terms,iterations,nodes,actualTerms,problem number,instance,perturbation,degree


In [27]:
for gen in df_map:
    mask = (-1e20 > df_map[gen]["lpBound"]) | \
        (df_map[gen]["lpBound"] - 1e-4 > df_map[gen]["lpBoundPostVpc"]) | \
        (df_map[gen]["lpBoundPostVpc"] - 1e-4 > df_map[gen]["disjunctiveDualBound"]) | \
        (df_map[gen]["lpBoundPostVpc"] - 1e-4 > df_map[gen]["rootDualBound"]) | \
        (df_map[gen]["rootDualBound"] - 1e-4 > df_map[gen]["dualBound"]) | \
        (df_map[gen]["dualBound"] - 1e-4 > df_map[gen]["primalBound"]) | \
        (df_map[gen]["primalBound"] > 1e20) | \
        (0 > df_map[gen]["vpcGenerationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-4 > df_map[gen]["heuristicTime"]) | \
        (df_map[gen]["heuristicTime"] - 1e-4 > df_map[gen]["rootDualBoundTime"]) | \
        (df_map[gen]["rootDualBoundTime"] - 1e-4 > df_map[gen]["terminationTime"]) | \
        (0 > df_map[gen]["firstSolutionTime"]) | \
        (df_map[gen]["firstSolutionTime"] - 1e-4 > df_map[gen]["bestSolutionTime"]) | \
        (df_map[gen]["bestSolutionTime"] - 1e-4 > df_map[gen]["terminationTime"]) | \
        (df_map[gen]["terminationTime"] > df_map[gen]["maxTime"] + 10) | \
        (df_map[gen]["vpcGenerationTime"] > df_map[gen]["maxTime"])
    df_map[gen] = df_map[gen][~mask]

In [41]:
# merge the 4 different data frames into one
join_cols = ["instance", "perturbation", "degree", "terms", "problem number"]
df = df_map["None"].merge(df_map["New"], on=join_cols,
                                suffixes=(" None", None))
df = df.merge(df_map["Old"], on=join_cols,
                          suffixes=(" New", None))
df = df.merge(df_map["Farkas"], on=join_cols,
                          suffixes=(" Old", " Farkas"))
df

,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,heuristicPrimalBound None,primalBound None,vpcGenerationTime None,heuristicTime None,rootDualBoundTime None,...,heuristicTime Farkas,rootDualBoundTime Farkas,firstSolutionTime Farkas,bestSolutionTime Farkas,terminationTime Farkas,maxTime Farkas,vpcGenerator Farkas,iterations Farkas,nodes Farkas,actualTerms Farkas
0,134.915590,134.915590,134.915590,143.653501,145.015871,154.000000,151.000000,0.096006,1.206488,3.890045,...,4.556782,7.583520,4.556799,12.220595,300.515738,300.0,New,190757,7909,4
1,135.862243,135.862243,135.862243,144.515640,144.515640,153.000000,149.000000,0.094931,1.026769,4.449761,...,1.190663,4.470498,1.190665,4.573789,301.040320,300.0,Farkas,182442,9874,4
2,135.354844,135.354844,135.354844,143.841106,145.807495,152.000000,150.000000,0.091691,1.041764,4.332778,...,1.237440,4.343022,1.237443,4.480485,301.141948,300.0,Farkas,137678,11000,4
3,135.311224,135.311224,135.311224,142.730884,143.890196,151.000000,149.000000,0.091364,1.693356,5.167685,...,1.897233,5.480878,1.897237,15.567069,301.455145,300.0,Farkas,171474,11332,4
4,134.405958,134.405958,134.405958,141.990942,143.758420,153.000000,148.000000,0.089762,1.678831,4.389748,...,1.879234,4.473597,1.879235,5.083431,300.528704,300.0,Farkas,178901,8271,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,38893.442891,38893.442891,38893.442891,38893.442891,39378.409831,40560.054142,40005.054142,0.009592,0.430366,0.443382,...,0.967708,1.009255,0.967719,1.019741,300.632495,300.0,Farkas,315206,50927,65
997,26511.178608,26511.178608,26511.178608,26511.178608,27104.457594,28593.617817,27638.141720,0.016177,1.124101,1.139054,...,1.723236,1.738186,1.723240,106.677015,300.846082,300.0,Farkas,515262,86552,65
998,51687.033613,51687.033613,51687.033613,51687.033613,52023.712306,54752.387825,52771.123829,0.013327,0.691344,0.704919,...,1.322108,1.335609,1.322114,205.004800,301.060260,300.0,Farkas,579314,97852,65
999,38893.310241,38893.310241,38893.310241,38893.310241,39377.750503,40559.141700,40004.141700,0.009793,0.435453,0.448441,...,0.981189,1.022604,0.981194,7.611164,300.663957,300.0,Farkas,326375,52953,65


In [42]:
# get proportion of tests run
4 * len(df) / count_instances

0.5823152995927865

In [43]:
def gap_closed(df, col):
    return (df[col] - df["lpBound None"]) / \
        (df['primalBound None'] - df["lpBound None"]) 

In [44]:
# find the optimality gap closed by each generator
df["Disjunction (New)"] = gap_closed(df, "disjunctiveDualBound New")
df["Disjunction (Old)"] = gap_closed(df, "disjunctiveDualBound Old")
df["VPCs (New)"] = gap_closed(df, "lpBoundPostVpc New")
df["VPCs (Old)"] = gap_closed(df, "lpBoundPostVpc Old")
df["VPCs (Farkas)"] = gap_closed(df, "lpBoundPostVpc Farkas")
df["Root Cuts (None)"] = gap_closed(df, "rootDualBound None")
df["Root Cuts (New)"] = gap_closed(df, "rootDualBound New")
df["Root Cuts (Old)"] = gap_closed(df, "rootDualBound Old")
df["Root Cuts (Farkas)"] = gap_closed(df, "rootDualBound Farkas")

In [32]:
# set aside core columns and filter for all subsequent dataframes
group_cols = ["instance", "perturbation", "degree", "terms"]
id_cols = ["problem number"]
mask = (df["problem number"] > 0) & (df["Disjunction (New)"] < .9999)

## Make bound table

In [33]:
# additional filtering for dataframe on bounds
fields = ["Disjunction (New)", "Disjunction (Old)", "VPCs (New)", "VPCs (Old)",
          "VPCs (Farkas)", "Root Cuts (None)", "Root Cuts (New)", "Root Cuts (Old)",
          "Root Cuts (Farkas)"]
bound_df = df.loc[mask, group_cols + id_cols + fields]
bound_df

,instance,perturbation,degree,terms,problem number,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Old),VPCs (Farkas),Root Cuts (None),Root Cuts (New),Root Cuts (Old),Root Cuts (Farkas)
1,neos-3610173-itata,rhs,2.0,4,1,0.095260,0.067357,0.000000,0.024850,0.024850,0.658666,0.645240,0.675501,0.669352
2,neos-3610173-itata,rhs,2.0,4,2,0.169384,0.062536,0.000000,0.008756,0.023759,0.579459,0.591980,0.577645,0.700866
3,neos-3610173-itata,rhs,2.0,4,3,0.067271,0.080286,0.000000,0.025113,0.025113,0.542025,0.622162,0.548271,0.569447
4,neos-3610173-itata,rhs,2.0,4,4,0.065977,0.065020,0.003172,0.017348,0.017348,0.557964,0.570849,0.577828,0.575895
5,neos-3610173-itata,rhs,2.0,4,5,0.114146,0.081629,0.013967,0.026777,0.026777,0.677859,0.689113,0.628409,0.629432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,mas76,rhs,1.0,64,6,0.133031,0.133031,0.119257,0.119169,0.131210,0.000000,0.119257,0.119169,0.131210
996,mas76,rhs,1.0,64,7,0.133013,0.133013,0.131128,0.131133,0.131165,0.000000,0.131128,0.131133,0.131165
997,mas76,rhs,1.0,64,8,0.133671,0.021046,0.130681,0.020953,0.000000,0.000000,0.130681,0.020953,0.000000
998,mas76,rhs,1.0,64,9,0.077459,0.000742,0.071608,0.000593,0.000000,0.000000,0.071608,0.000593,0.000000


In [34]:
aggregations = {f: "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

In [35]:
# get gap closed by degree and term
out = bound_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table.csv"), index=False, mode="w")
out

,degree,terms,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Old),VPCs (Farkas),Root Cuts (None),Root Cuts (New),Root Cuts (Old),Root Cuts (Farkas),instance,problem number
0,0.5,4,0.073791,0.060983,0.062780,0.051739,0.040636,0.209070,0.215763,0.218170,0.214809,5,75
1,0.5,16,0.205294,0.154929,0.177432,0.134458,0.142134,0.148350,0.222567,0.203945,0.198334,4,46
2,0.5,64,0.297801,0.161775,0.215310,0.128260,0.129062,0.156051,0.274371,0.243686,0.240480,4,32
3,1.0,4,0.097033,0.078607,0.041203,0.041055,0.036043,0.406271,0.419587,0.415470,0.409310,11,158
4,1.0,16,0.223158,0.165781,0.110703,0.102995,0.094399,0.396335,0.431415,0.427759,0.423479,9,115
5,1.0,64,0.295585,0.202876,0.148824,0.123580,0.106709,0.351800,0.424974,0.418049,0.412570,8,65
6,2.0,4,0.098286,0.080080,0.040835,0.038570,0.030918,0.423796,0.430226,0.437096,0.426722,11,163
7,2.0,16,0.229777,0.170339,0.101361,0.096245,0.082861,0.437452,0.463841,0.452121,0.447142,9,126
8,2.0,64,0.253268,0.152833,0.113004,0.073353,0.037973,0.348343,0.386804,0.377266,0.367054,9,60


In [36]:
# now break it down by type of perturbation
bound_df.groupby(["degree", "terms", "perturbation"]).agg(aggregations)

Disjunction (New)  Disjunction (Old)  VPCs (New)  \
degree terms perturbation                                                     
0.5    4     matrix                 0.064312           0.054201    0.065451   
             objective              0.161299           0.142478    0.125629   
             rhs                    0.047538           0.033804    0.031286   
       16    matrix                 0.235694           0.200318    0.227046   
             objective              0.620113           0.328254    0.424711   
             rhs                    0.069669           0.063939    0.063517   
       64    matrix                 0.267250           0.168922    0.258480   
             objective              0.829368           0.332815    0.535242   
             rhs                    0.153198           0.111205    0.113622   
1.0    4     matrix                 0.090236           0.071212    0.033752   
             objective              0.190737           0.151047    0.121156   
             rhs                    0.091251           0.076654    0.038211   
       16    matrix                 0.229707           0.160063    0.100550   
             objective              0.526151           0.292795    0.233213   
             rhs                    0.177331           0.155035    0.104975   
       64    matrix                 0.377084           0.283599    0.208499   
             objective              0.726392           0.302266    0.338158   
             rhs                    0.180181           0.127079    0.079159   
2.0    4     matrix                 0.103629           0.075371    0.035310   
             objective              0.145718           0.107432    0.078389   
             rhs                    0.084909           0.079457    0.039123   
       16    matrix                 0.220584           0.164146    0.081330   
             objective              0.529136           0.296017    0.231798   
             rhs                    0.189994           0.155601    0.097508   
       64    matrix                 0.314079           0.208064    0.115065   
             objective              0.813897           0.367941    0.569640   
             rhs                    0.192065           0.105895    0.098058   

                           VPCs (Old)  VPCs (Farkas)  Root Cuts (None)  \
degree terms perturbation                                                
0.5    4     matrix          0.053346       0.044396          0.179617   
             objective       0.104270       0.070231          0.611309   
             rhs             0.026249       0.022469          0.069568   
       16    matrix          0.193769       0.185949          0.163773   
             objective       0.190914       0.302751          0.674395   
             rhs             0.058067       0.055975          0.000644   
       64    matrix          0.164792       0.152089          0.069757   
             objective       0.178440       0.184694          0.652244   
             rhs             0.105224       0.107685          0.034828   
1.0    4     matrix          0.030098       0.028487          0.438365   
             objective       0.136100       0.073348          0.649405   
             rhs             0.039968       0.039446          0.332759   
       16    matrix          0.085210       0.073182          0.454020   
             objective       0.184843       0.245726          0.633455   
             rhs             0.110169       0.095998          0.307912   
       64    matrix          0.193535       0.159515          0.439279   
             objective       0.166668       0.166668          0.699582   
             rhs             0.062958       0.057721          0.241415   
2.0    4     matrix          0.030151       0.028779          0.433201   
             objective       0.086473       0.049383          0.546915   
             rhs             0.037635       0.029552          0.393136   
       16    matrix          0.08

## Make time table

In [37]:
def optimality_gap(df, generator):
    return abs(df[f"primalBound {generator}"] - df[f"dualBound {generator}"]) / \
        (df[f"primalBound {generator}"])

In [38]:
# additional filtering for dataframe on run time
fields = ["terminationTime New", "terminationTime Old",
          "terminationTime Farkas", "terminationTime None",
          "vpcGenerationTime New", "vpcGenerationTime Old",
          "vpcGenerationTime Farkas"]
mask = mask & (optimality_gap(df, "New") < .02)  # only check the ones that solve to optimality
time_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
out = time_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "time.csv"), index=False, mode="w")
out

,degree,terms,terminationTime New,terminationTime Old,terminationTime Farkas,terminationTime None,vpcGenerationTime New,vpcGenerationTime Old,vpcGenerationTime Farkas,instance,problem number
0,0.5,4,16.355662,12.822757,11.038034,13.778225,2.760208,0.687757,0.192316,5,25
1,0.5,16,22.791149,7.971118,4.462865,4.576061,17.250001,3.645910,0.614967,3,16
2,0.5,64,118.495040,73.594722,52.840414,50.518878,82.424231,27.025128,3.577249,2,7
3,1.0,4,91.669882,89.162975,86.463025,82.904498,4.618980,0.644236,0.258162,6,51
4,1.0,16,60.049696,47.124476,43.035859,42.464663,19.521022,3.709673,1.365795,4,32
5,1.0,64,31.664069,14.954202,10.414000,5.775945,25.818715,9.426301,1.523261,2,10
6,2.0,4,59.051592,60.142018,64.818682,59.409460,3.634760,0.603724,0.229709,7,48
7,2.0,16,26.833795,10.408978,6.743516,6.089188,17.027402,2.894695,0.664367,2,30
8,2.0,64,21.109950,9.055443,3.471070,2.859631,18.662018,6.749112,1.342633,2,4


# Make node table

In [39]:
# additional filtering for dataframe on nodes processed
fields = ["nodes New", "nodes Old", "nodes Farkas",
          "nodes None"]
node_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
out = node_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "node.csv"), index=False, mode="w")
out

,degree,terms,nodes New,nodes Old,nodes Farkas,nodes None,instance,problem number
0,0.5,4,775.480000,621.600000,495.360000,1131.640000,5,25
1,0.5,16,176.625000,155.625000,167.250000,178.375000,3,16
2,0.5,64,1665.428571,2177.142857,2145.857143,1830.000000,2,7
3,1.0,4,3989.254902,3940.745098,3659.784314,3843.137255,6,51
4,1.0,16,1317.750000,1565.718750,1566.906250,1535.468750,4,32
5,1.0,64,271.200000,287.900000,434.600000,244.800000,2,10
6,2.0,4,2143.541667,2338.750000,2621.708333,2573.312500,7,48
7,2.0,16,251.100000,307.333333,238.733333,248.733333,2,30
8,2.0,64,205.750000,178.500000,146.000000,170.500000,2,4


# Make iteration table

In [40]:
# additional filtering for dataframe on nodes processed
fields = ["iterations New", "iterations Old",
          "iterations Farkas", "iterations None"]
node_df = df.loc[mask, group_cols + id_cols + fields]

# define aggregating operations
aggregations = {f: "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["problem number"] = "count"

# get gap closed by degree and term
out = node_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "iteration.csv"), index=False, mode="w")
out

,degree,terms,iterations New,iterations Old,iterations Farkas,iterations None,instance,problem number
0,0.5,4,10795.840000,9656.160000,8158.680000,11932.640000,5,25
1,0.5,16,3284.937500,3115.625000,2957.812500,3321.812500,3,16
2,0.5,64,22263.285714,27118.571429,34794.571429,36691.714286,2,7
3,1.0,4,46896.745098,46816.333333,45516.784314,44171.470588,6,51
4,1.0,16,17698.250000,21021.500000,18389.875000,18939.093750,4,32
5,1.0,64,4438.900000,4906.500000,7611.800000,4616.700000,2,10
6,2.0,4,26166.312500,27887.979167,31883.625000,30375.229167,7,48
7,2.0,16,5669.166667,6484.066667,4889.333333,5219.433333,2,30
8,2.0,64,2452.750000,2301.250000,2189.750000,2535.000000,2,4
